In [1]:
import numpy as np
import getopt
import os, sys
 
# Main path to your caffe installation
caffe_root = '/home/payam/workspace/caffe/'

sys.path.insert(0, caffe_root + 'python')
import caffe
 
# Model prototxt file
#model_prototxt = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
model_prototxt = caffe_root + 'models/bvlc_googlenet/deploy.prototxt'
 
# Model caffemodel file
#model_trained = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
model_trained = caffe_root + 'models/bvlc_googlenet/bvlc_googlenet.caffemodel'
 
# File containing the class labels
imagenet_labels = caffe_root + 'data/ilsvrc12/synset_words.txt'
 
# Path to the mean image (used for input processing)
mean_path = caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy'
 


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Name of the layer we want to extract
#layer_name = 'fc8'
layer_name = 'pool5/7x7_s1'
layer_name2 = 'loss3/classifier'
layer_name3 = 'pool5/drop_7x7_s1'
def folder_feature_extractor(input_folder):
    caffe.set_mode_cpu()
    net = caffe.Classifier(model_prototxt, model_trained,
                           mean=np.load(mean_path).mean(1).mean(1),
                           channel_swap=(2,1,0),
                           raw_scale=255,
                           image_dims=(256, 256))
 
    with open(imagenet_labels) as f:
        labels = f.readlines()
    feature_list = []
    feature_list2 = []
    feature_list3 = []
    for i in range(1,101):
        net = caffe.Classifier(model_prototxt, model_trained,
                           mean=np.load(mean_path).mean(1).mean(1),
                           channel_swap=(2,1,0),
                           raw_scale=255,
                           image_dims=(256, 256))
        file_name =  str(i).zfill(3) + '.jpg'
        file_name = input_folder + file_name
        #print file_name
        image = caffe.io.load_image(file_name)
        #plt.imshow(image)
        #plt.figure(i+1)
        prediction = net.predict([image], oversample=False)
        features = net.blobs[layer_name].data[0].reshape(1,-1)
        features2 = net.blobs[layer_name2].data[0].reshape(1,-1)
        #features3 = net.blobs[layer_name3].data[0].reshape(1,-1)
        #print sum(features[0])
        #print labels[prediction[0].argmax()].strip() , ' (', prediction[0][prediction[0].argmax()] , ')'
        #for i in features[0]:
        #    print i
        #print type(features)
        #print len(features)
        feature_list.append(features[0])
        feature_list2.append(features2[0])
        #feature_list3.append(features3[0])
        #print '-------------------------------------'
    return [feature_list, feature_list2]

ref1 = folder_feature_extractor('/home/payam/dataset/cd_covers/Reference/')[0]
ref2 = folder_feature_extractor('/home/payam/dataset/cd_covers/Reference/')[1]
print ref1[0].shape
print ref2[0].shape
test1 = folder_feature_extractor('/home/payam/dataset/cd_covers/Droid/')[0]
test2 = folder_feature_extractor('/home/payam/dataset/cd_covers/Droid/')[1]
print test1[0].shape
print test2[0].shape

In [ ]:
ref = np.loadtxt('/home/payam/dataset/cd_covers/Reference/caffenet_features.txt')
test = np.loadtxt('/home/payam/dataset/cd_covers/Droid/caffenet_features.txt')

In [ ]:
from scipy.stats.stats import pearsonr
count_dist = 0
count_corr = 0
count_coef = 0
count_pear = 0
count_comb = 0
print 'ind   dist   corr   pear   comb'
for j in range(0,100):
    min_dist = 9999999999.0
    min_index = -1
    max_coef = 0.0
    max_coef_index = -1
    max_corr = 0.0
    max_corr_index = -1
    max_pear = 0.0
    max_pear_index = -1
    for i in range(0,100):
        coef = np.mean(np.corrcoef(ref[i], test[j]))
        corr = np.correlate(ref[i], test[j])
        dist = np.linalg.norm(ref[i] - test[j])
        pear = abs(pearsonr(ref[i], test[j])[0])
        if dist < min_dist:
            min_index = i
            min_dist = dist
        if corr > max_corr:
            max_corr_index = i
            max_corr = corr
        if pear > max_pear:
            max_pear_index = i
            max_pear = pear
    #print '%3d %6d %6d %6d '%(j, min_index, max_corr_index, max_pear_index)
    flag = False
    #count = 0
    comb = np.zeros((100))
    comb[min_index] += 39
    comb[max_corr_index] += 21
    comb[max_pear_index] += 44
    comb_index = np.argmax(comb)
    if comb_index == j:
        count_comb += 1
        flag = True
    if j == min_index:
        count_dist += 1
        flag = True
    if j == max_corr_index:
        count_corr += 1
        flag = True
    if j == max_pear_index:
        count_pear += 1
        flag = True
    if flag:
        pass
        #print '%3d %6d %6d %6d %6d'%(j, min_index, max_corr_index, max_pear_index, comb_index)
    
print 'distance:    ', count_dist
print 'correlation: ', count_corr
#print 'coef:        ', count_coef
print 'pearson    : ', count_pear
print 'combination: ', count_comb